# Ionic using LangChain's OpenAI Tools Agent

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ioniccommerce/examples/blob/main/examples/notebooks/langchain_openai_tools.ipynb)

[Ionic](https://www.ioniccommerce.com/) is a plug and play ecommerce marketplace for AI Assistants. By using Ionic, you are effortlessly providing your users with the ability to shop and transact directly within your agent, and you'll get a cut of the transaction.

This is a basic jupyter notebook demonstrating how to integrate the Ionic Shopping Tool. For more information on setting up your Agent with Ionic, see the Ionic [documentation](https://docs.ioniccommerce.com/introduction).

This Jupyter Notebook demonstrates how to use Ionic with LangChain's OpenAI Tools Agent.

### Install dependencies

In [ ]:
%%bash

pip install --upgrade pip
pip install langchain langchain_openai langchainhub
pip install ionic-langchain

### Setup OpenAI Key
Create an API key at [OpenAI | API Keys](https://platform.openai.com/api-keys)

Either set your environment variable: `export OPENAI_API_KEY=<YOUR_API_KEY_HERE` 

Or edit the following code block and replace "YOUR_API_KEY_HERE" with your key. Be careful to not leak your API Key.

In [7]:
import os

openai_api_key = os.environ.get("OPENAI_API_KEY", "YOUR_API_KEY_HERE") 

### Create your Agent with the Ionic Commerce Tool

In [8]:
from ionic_langchain.tool import IonicTool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

# Based on OpenAI Tools Agent
# https://python.langchain.com/docs/modules/agents/agent_types/openai_tools

model = "gpt-3.5-turbo-1106"
temperature = 0.6

# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model=model, temperature=temperature, openai_api_key=openai_api_key)

ionic_tool = IonicTool().tool()

# The tool comes with its own prompt,
# but you may also update it directly via the description attribute:
ionic_tool.description = str(
    """
Ionic is an e-commerce shopping tool. Assistant uses the Ionic Commerce Shopping Tool to find, discover, and compare products from thousands of online retailers. Assistant should use the tool when the user is looking for a product recommendation or trying to find a specific product.

The user may specify the number of results, minimum price, and maximum price for which they want to see results.
Ionic Tool input is a comma-separated string of values:
  - query string (required, must not include commas)
  - number of results (default to 4, no more than 10)
  - minimum price in cents ($5 becomes 500)
  - maximum price in cents
For example, if looking for coffee beans between 5 and 10 dollars, the tool input would be `coffee beans, 5, 500, 1000`.

Return them as a markdown formatted list with each recommendation from tool results, being sure to include the full PDP URL. For example:

1. Product 1: [Price] -- link
2. Product 2: [Price] -- link
3. Product 3: [Price] -- link
4. Product 4: [Price] -- link
"""
)

tools = [ionic_tool]

# default prompt for openai tools agent
prompt = hub.pull("hwchase17/openai-tools-agent")

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True, max_iterations=5
)

### Run the Agent

In [9]:
input = (
    "I'm looking for a new 4k monitor can you find me some options for less than $1000"
)
agent_executor.invoke({"input": input})



> Entering new AgentExecutor chain...

Invoking: `ionic_commerce_shopping_tool` with `4k monitor, 4, 0, 100000`


[{'products': [{'links': [{'text': 'Details', 'type': 'pdp', 'url': 'https://www.amazon.com/dp/B098HQ8YRX?tag=ioniccommer00-20&linkCode=osi&th=1&psc=1'}], 'merchant_name': 'Amazon', 'merchant_product_id': 'B098HQ8YRX', 'name': 'Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor up to 70Hz DisplayPort HDMI 99% sRGB Build-in Speakers, Black 2021 (U275W-UPT)', 'price': '$169.97', 'status': 'available', 'thumbnail': 'https://m.media-amazon.com/images/I/41URcXrndAL._SL160_.jpg', 'brand_name': 'SCEPTRE', 'upc': None}, {'links': [{'text': 'Details', 'type': 'pdp', 'url': 'https://brwi.short.gy/S1B0J1'}], 'merchant_name': 'Walmart', 'merchant_product_id': '2624247367', 'name': 'Sceptre 32-inch IPS 4K UHD 3840 x 2160 Monitor DisplayPort x2 HDMI x2 99% sRGB Build-in Speakers, Machine Black 2024(U325W-UPT)', 'price': '$199.97', 'status': 'available', 'thumbnail': 'https://i5.walmartimages.c

{'input': "I'm looking for a new 4k monitor can you find me some options for less than $1000",
 'output': 'Here are some options for 4K monitors under $1000:\n\n1. Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor up to 70Hz DisplayPort HDMI 99% sRGB Build-in Speakers, Black 2021 (U275W-UPT) - $169.97\n   [Details](https://www.amazon.com/dp/B098HQ8YRX?tag=ioniccommer00-20&linkCode=osi&th=1&psc=1)\n\n2. Sceptre 32-inch IPS 4K UHD 3840 x 2160 Monitor DisplayPort x2 HDMI x2 99% sRGB Build-in Speakers, Machine Black 2024(U325W-UPT) - $199.97\n   [Details](https://brwi.short.gy/S1B0J1)\n\n3. Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor up to 70Hz DisplayPort HDMI 99% sRGB Build-in Speakers, Machine Black (U275W-UPT) - $169.97\n   [Details](https://brwi.short.gy/U7ovsY)\n\n4. SAMSUNG UJ59 Series 32-Inch 4K UHD (3840x2160) Computer Monitor, HDMI, Display Port, Eye Saver/Flicker Free Mode, FreeSync (LU32J590UQNXZA),Black - $279.99\n   [Details](https://www.amazon.com/dp/B07CS3JGPC?tag=ioniccommer00-20&